# Using different Vizier Catalogues in prediction() and Star()

As the current most precise catalog available, Gaia is the default catalog implemented in SORA. Thus, the user can make predictions or adding a star in the occultation procedure by providing the coordinates or the Gaia source ID of the targets. The user can also choose between Gaia-DR2 and Gaia-EDR3 as the source catalog. However, Gaia does not have the brightest stars on the sky in its list, which can be found in Hipparcos, for instance. In other cases, some of the faintest stars are also not available, and may be available in other catalogs.

Regarding this problem, from SORA v0.3 onwards can allow the user to implement any new catalog available on Vizier that can be used in the prediciton() function and the Star() class.

To add a new catalog, the user has to provide the keywords for the necessary parameters allowing SORA to retrieve the correct values.

In [1]:
from sora.star.catalog import VizierCatalogue
from astropy.time import Time
from sora import Star

SORA version: 0.3


In [2]:
# Defining the NOMAD catalogue from Vizier
nomad = VizierCatalogue(name='NOMAD', cat_path='I/297/out', code='NOMAD1', ra='RAJ2000', dec='DEJ2000',
                        epoch=Time('J2000'), band={'B': 'Bmag', 'V': 'Vmag', 'R': 'Rmag', 'J': 'Jmag', 'H': 'Hmag', 'K': 'Kmag'})

Note the parameters `name` (name of the catalogue) and `cat_path` (path of the catalogue on Vizier) are required parameters for the proper identification of the catalog. The parameters `code`, `ra`, `dec` and `epoch` are required for a proper identification of the star. The `code` parameter is a unique identifier within the catalog. In Gaia, for instanca, `code="Source"`.

The parameters for the other astrometric parameters can also be given, like `pmra` and `pmdec` for the proper motions PM_RA\*COS(DEC) and PM_DEC, `parallax` and `rad_vel` for parallax and radial velocity. If not given, these parameters are default to zero. Finally, if the user wants to characterize the stars' magnitudes, the `band` parameter can be given by a dictionary with the keywords for all the bands available, as shown for the NOMAD catalog above.

### Predicting with new catalog

To predict with a different catalogue than Gaia, just pass the new created VizierCatalogue object to the parameter `catalogue` in the prediction() function

In [3]:
from sora import Body
from sora.prediction import prediction

mars = Body(name='Mars', orbit_class='planet', diameter=6779, ephem='horizons', spkid='499', database=None)

p = prediction(body=mars, time_beg='2022-09-10 00:00', time_end='2022-09-10 06:00', catalogue=nomad)
p.pprint_all()

Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2022-09-10 00:00:00.000 and 2022-09-10 05:59:00.000 ...
    54 NOMAD stars downloaded
Identifying occultations ...

12 occultations found.
         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    B      V      R      J      H      K    long  loct M-G-T S-G-T NOMAD Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag    mag    mag    mag    mag   deg  hh:mm  deg   deg                 
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ------ ------ ------ ------ ---- ----- ----- ----- ---------------
2022-09-10 00:37:10.480 04 37 55.92334 +20 56 19.88016 04 37 55.84499 +20 56 25.91258  6.131 349.69  13.46   0.906 20.750    nan 19.880    nan    nan 

If `catalogue` is not given, then the default catalogue is the Gaia-EDR3. The `catalogue` parameter can also be "gaiadr2" or "gaiaedr3" as it has been the case since SORA v0.2.

Note that all the bands given to the nomad object were downloaded and included in the PredictionTable. Those stars where a given band is not defined is shown with "Not A Number" (nan) mark.

In order to restrict the prediction function based on a magnitude limit, the user has to define the `mag_lim` parameter in the prediction() function. Previously to SORA v0.3, only a number could be given to the parameter. Currently, if a number is passed, the limit of magnitude will be defined only for the first band defined. In this example, it is the "B" band. To define which bands are to be limited, the user must pass a dictionary to the `mag_lim` parameter. For instance, the next piece of code predicts occultations limiting stars with the V band smaller than 16 and the H band smaller than 14.

In [4]:
p = prediction(body=mars, time_beg='2022-09-10', time_end='2022-09-11', catalogue=nomad, mag_lim={'V': 16, 'H': 14})
p.pprint_all()

Ephemeris was split in 1 parts for better search of stars

Searching occultations in part 1/1
Generating Ephemeris between 2022-09-10 00:00:00.000 and 2022-09-10 23:59:00.000 ...
    39 NOMAD stars downloaded
Identifying occultations ...

4 occultations found.
         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    B      V      R      J      H      K    long  loct M-G-T S-G-T NOMAD Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag    mag    mag    mag    mag   deg  hh:mm  deg   deg                 
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ------ ------ ------ ------ ---- ----- ----- ----- ---------------
2022-09-10 02:37:08.420 04 38 06.11633 +20 56 57.82092 04 38 06.18842 +20 56 52.26594  5.646 169.70  13.44   0.906 16.350 15.490 16.100 13.865 13.381 1

### Using a not Gaia star in the occultation process

To include a star in the occultation procedure, we must use the Star() class to define a single star. The standard catalogs defined in this class are Gaia-DR2 and Gaia-EDR3. However, we can add a star from a different catalogue defined in Vizier using the VizierCatalogue() class just like we did in the prediction() function. Notice that, in this case, we are not able to use the shortcut of the Occultation() class by providing the direct coordinate of the star. We must define a Star object previously.

In [5]:
# The Star class already reads the nomad catalogue to download the magnitudes in the available bands.
# In the case of this example, we must set it to False, or it will download the bands as the main catalogue
# and the auxiliary one, replacing the magnitude with the same values.
s = Star(code='1109-0056199', catalogue=nomad, nomad=False)
print(s)

1 NOMAD star found band={'B': 14.069999694824219, 'V': 13.25, 'R': 13.479999542236328, 'J': 10.62399959564209, 'H': 9.95199966430664, 'K': 9.793000221252441}
star coordinate at J2000.0: RA=4h38m10.97573s +/- 0.0 mas, DEC=20d57m01.7089s +/- 0.0 mas
NOMAD star Source ID: 1109-0056199
ICRS star coordinate at J2000.0:
RA=4h38m10.97573s +/- 0.0000 mas, DEC=20d57m01.7089s +/- 0.0000 mas
pmRA=0.000 +/- 0.000 mas/yr, pmDEC=0.000 +/- 0.000 mas/yr
Plx=0.0000 +/- 0.0000 mas, Rad. Vel.=0.00 +/- 0.00 km/s 

Magnitudes: B: 14.070, V: 13.250, R: 13.480, J: 10.624, H:  9.952, K:  9.793

Apparent diameter from Kervella et. al (2004):
    V: 0.0660 mas, B: 0.0611 mas
Apparent diameter from van Belle (1999):
    sg: B: 0.0596 mas, V: 0.0616 mas
    ms: B: 0.0844 mas, V: 0.0579 mas
    vs: B: 0.0848 mas, V: 0.0781 mas


Usually, when working with the occultation we want to estimate the uncertainty on the position of the star at the epoch of occultation. For this, the user can provide the list of keywords for the uncertainty of each of the astrometric parameters (if not available, None must be passed for the parameter uncertainty). An important note is that only with the Gaia catalog SORA will obtain the full covariance matrix.

In [6]:
nomad = VizierCatalogue(name='NOMAD', cat_path='I/297/out', code='NOMAD1', ra='RAJ2000', dec='DEJ2000', pmra='pmRA', pmdec='pmDE',
                        epoch=Time('J2000'), band={'B': 'Bmag', 'V': 'Vmag', 'R': 'Rmag', 'J': 'Jmag', 'H': 'Hmag', 'K': 'Kmag'},
                        errors=['e_RAJ2000', 'e_DEJ2000', 'e_pmRA', 'e_pmDE', None, None])

s = Star(code='1109-0056199', catalogue=nomad, verbose=False, nomad=False)
print(s)

NOMAD star Source ID: 1109-0056199
ICRS star coordinate at J2000.0:
RA=4h38m10.97573s +/- 31.0000 mas, DEC=20d57m01.7089s +/- 17.0000 mas
pmRA=-7.100 +/- 6.400 mas/yr, pmDEC=-16.600 +/- 6.300 mas/yr
Plx=0.0000 +/- 0.0000 mas, Rad. Vel.=0.00 +/- 0.00 km/s 

Magnitudes: B: 14.070, V: 13.250, R: 13.480, J: 10.624, H:  9.952, K:  9.793

Apparent diameter from Kervella et. al (2004):
    V: 0.0660 mas, B: 0.0611 mas
Apparent diameter from van Belle (1999):
    sg: B: 0.0596 mas, V: 0.0616 mas
    ms: B: 0.0844 mas, V: 0.0579 mas
    vs: B: 0.0848 mas, V: 0.0781 mas


In [7]:
from sora import Occultation

occ = Occultation(star=s, body=mars, time='2022-09-10 03:32')
print(occ)

Stellar occultation of star NOMAD 1109-0056199 by Mars.

Geocentric Closest Approach: 3.042 arcsec
Instant of CA: 2022-09-10 03:32:06.900
Position Angle: 349.70 deg
Geocentric shadow velocity: 13.43 km / s
Sun-Geocenter-Target angle:  96.16 deg
Moon-Geocenter-Target angle: 87.36 deg


No observations reported

###############################################################################
                                     STAR                                      
###############################################################################
NOMAD star Source ID: 1109-0056199
ICRS star coordinate at J2000.0:
RA=4h38m10.97573s +/- 31.0000 mas, DEC=20d57m01.7089s +/- 17.0000 mas
pmRA=-7.100 +/- 6.400 mas/yr, pmDEC=-16.600 +/- 6.300 mas/yr
Plx=0.0000 +/- 0.0000 mas, Rad. Vel.=0.00 +/- 0.00 km/s 

Magnitudes: B: 14.070, V: 13.250, R: 13.480, J: 10.624, H:  9.952, K:  9.793

Apparent diameter from Kervella et. al (2004):
    V: 0.0660 mas, B: 0.0611 mas
Apparent diameter from van Belle 